In [ ]:
# Import des bibliothèques nécessaires
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
import sys
import joblib

from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix
from sklearn.metrics import classification_report, roc_curve, auc, roc_auc_score

In [ ]:
# Ajouter le répertoire parent au path pour importer les fonctions du projet
sys.path.append('..')
from src.preprocessing import load_data, clean_data, create_preprocessing_pipeline
from src.model import train_model, evaluate_model, plot_feature_importance

In [ ]:
# Configuration matplotlib et seaborn
plt.style.use('seaborn-v0_8-whitegrid')
sns.set_palette('Blues_r')
plt.rcParams['figure.figsize'] = (12, 6)
plt.rcParams['font.size'] = 12

In [ ]:
# Supprimer les avertissements
import warnings
warnings.filterwarnings('ignore')

# Chemin vers le fichier de données
data_path = '../data/airline_satisfaction.csv'

In [ ]:
# Charger et nettoyer les données
df = load_data(data_path)
df_clean = clean_data(df)

# Afficher un aperçu des données préparées
print(f"Nombre total d'enregistrements: {len(df_clean)}")
df_clean.head()

In [ ]:
# Préparer les features et la cible
if 'satisfaction' in df_clean.columns:
    X = df_clean.drop('satisfaction', axis=1)
    y = df_clean['satisfaction']
    
    # Supprimer la colonne ID si elle existe
    if 'id' in X.columns:
        X = X.drop('id', axis=1)
    
    print(f"Forme des features (X): {X.shape}")
    print(f"Forme de la cible (y): {y.shape}")
    print(f"Répartition de la cible: {y.value_counts(normalize=True) * 100}")
else:
    print("Erreur: Colonne 'satisfaction' introuvable dans les données.")

In [ ]:
# Diviser les données en ensembles d'entraînement et de test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

print(f"Taille de l'ensemble d'entraînement: {X_train.shape[0]} échantillons")
print(f"Taille de l'ensemble de test: {X_test.shape[0]} échantillons")

In [ ]:
# Identifier les variables catégorielles et numériques
cat_features = X.select_dtypes(include=['object']).columns.tolist()
num_features = X.select_dtypes(include=['number']).columns.tolist()

print(f"Variables catégorielles: {cat_features}")
print(f"Variables numériques: {num_features}")

# Créer le pipeline de prétraitement
preprocessor = create_preprocessing_pipeline(X)

# Créer un pipeline complet avec le prétraitement et un modèle de base
model_pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('classifier', RandomForestClassifier(n_estimators=100, random_state=42))
])

In [ ]:
# Entraîner le modèle de base
model_pipeline.fit(X_train, y_train)

# Faire des prédictions sur l'ensemble de test
y_pred = model_pipeline.predict(X_test)

In [ ]:
# Évaluer le modèle
print("Évaluation du modèle de base (Random Forest):")
print(f"Accuracy: {accuracy_score(y_test, y_pred):.4f}")
print(f"Precision: {precision_score(y_test, y_pred):.4f}")
print(f"Recall: {recall_score(y_test, y_pred):.4f}")
print(f"F1 Score: {f1_score(y_test, y_pred):.4f}")

In [ ]:
# Afficher le rapport de classification
print("\nRapport de classification détaillé:")
print(classification_report(y_test, y_pred))

# Matrice de confusion
cm = confusion_matrix(y_test, y_pred)
plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues')
plt.title('Matrice de confusion')
plt.ylabel('Valeur réelle')
plt.xlabel('Valeur prédite')
plt.show()

In [ ]:
# Définir plusieurs modèles à comparer
models = {
    'Logistic Regression': LogisticRegression(max_iter=1000, random_state=42),
    'Random Forest': RandomForestClassifier(n_estimators=100, random_state=42),
    'Gradient Boosting': GradientBoostingClassifier(n_estimators=100, random_state=42)
}

# Stocker les résultats
results = []

for name, model in models.items():
    # Créer un pipeline avec le prétraitement et le modèle
    pipeline = Pipeline(steps=[
        ('preprocessor', preprocessor),
        ('classifier', model)
    ])
    
    # Entraîner le modèle
    pipeline.fit(X_train, y_train)
    
    # Faire des prédictions
    y_pred = pipeline.predict(X_test)
    
    # Calculer les métriques
    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred)
    recall = recall_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred)
    
    # Calculer l'AUC-ROC si le modèle supporte predict_proba
    if hasattr(model, 'predict_proba'):
        y_prob = pipeline.predict_proba(X_test)[:, 1]
        auc_roc = roc_auc_score(y_test, y_prob)
    else:
        auc_roc = None
    
    # Ajouter les résultats
    results.append({
        'Model': name,
        'Accuracy': accuracy,
        'Precision': precision,
        'Recall': recall,
        'F1 Score': f1,
        'AUC-ROC': auc_roc,
        'Pipeline': pipeline
    })


In [ ]:
# Afficher les résultats
results_df = pd.DataFrame(results).drop('Pipeline', axis=1)
results_df

# Visualiser les résultats
plt.figure(figsize=(14, 6))
metrics = ['Accuracy', 'Precision', 'Recall', 'F1 Score']

results_melted = pd.melt(results_df, id_vars=['Model'], value_vars=metrics, var_name='Metric', value_name='Score')
sns.barplot(x='Model', y='Score', hue='Metric', data=results_melted)
plt.title('Comparaison des modèles')
plt.ylabel('Score')
plt.ylim(0.7, 1.0)  # Ajuster selon vos résultats
plt.legend(title='Métrique')
plt.show()

# Courbes ROC pour tous les modèles
plt.figure(figsize=(10, 8))

for result in results:
    if result['AUC-ROC'] is not None:
        pipeline = result['Pipeline']
        y_prob = pipeline.predict_proba(X_test)[:, 1]
        fpr, tpr, _ = roc_curve(y_test, y_prob)
        plt.plot(fpr, tpr, label=f"{result['Model']} (AUC = {result['AUC-ROC']:.3f})")

plt.plot([0, 1], [0, 1], 'k--')
plt.xlabel('Taux de faux positifs')
plt.ylabel('Taux de vrais positifs')
plt.title('Courbes ROC')
plt.legend(loc='lower right')
plt.show()

In [ ]:
# Trouver le meilleur modèle
best_model_idx = results_df['F1 Score'].idxmax()
best_model_name = results_df.loc[best_model_idx, 'Model']
best_model_pipeline = results[best_model_idx]['Pipeline']

print(f"Le meilleur modèle est: {best_model_name}")

# Définir une grille de paramètres pour ce modèle
if best_model_name == 'Random Forest':
    param_grid = {
        'classifier__n_estimators': [50, 100, 200],
        'classifier__max_depth': [None, 10, 20],
        'classifier__min_samples_split': [2, 5, 10],
        'classifier__min_samples_leaf': [1, 2, 4]
    }
elif best_model_name == 'Gradient Boosting':
    param_grid = {
        'classifier__n_estimators': [50, 100, 200],
        'classifier__learning_rate': [0.01, 0.1, 0.2],
        'classifier__max_depth': [3, 5, 7]
    }
elif best_model_name == 'Logistic Regression':
    param_grid = {
        'classifier__C': [0.1, 1.0, 10.0],
        'classifier__solver': ['liblinear', 'lbfgs'],
        'classifier__penalty': ['l1', 'l2']
    }
    
# Effectuer une recherche par grille avec validation croisée
grid_search = GridSearchCV(
    best_model_pipeline,
    param_grid,
    cv=5,
    scoring='f1',
    n_jobs=-1,
    verbose=1
)

print(f"Recherche des meilleurs paramètres pour {best_model_name}...")
grid_search.fit(X_train, y_train)

print(f"Meilleurs paramètres: {grid_search.best_params_}")
print(f"Meilleur score de validation croisée: {grid_search.best_score_:.4f}")

In [ ]:
# Évaluer le modèle optimisé
best_model = grid_search.best_estimator_
y_pred_best = best_model.predict(X_test)

print("\nÉvaluation du modèle optimisé:")
print(f"Accuracy: {accuracy_score(y_test, y_pred_best):.4f}")
print(f"Precision: {precision_score(y_test, y_pred_best):.4f}")
print(f"Recall: {recall_score(y_test, y_pred_best):.4f}")
print(f"F1 Score: {f1_score(y_test, y_pred_best):.4f}")

In [ ]:
# Afficher le rapport de classification
print("\nRapport de classification détaillé:")
print(classification_report(y_test, y_pred_best))

In [ ]:
# Matrice de confusion
cm_best = confusion_matrix(y_test, y_pred_best)
plt.figure(figsize=(8, 6))
sns.heatmap(cm_best, annot=True, fmt='d', cmap='Blues')
plt.title('Matrice de confusion du modèle optimisé')
plt.ylabel('Valeur réelle')
plt.xlabel('Valeur prédite')
plt.show()

In [ ]:
# Importance des variables (si le modèle le supporte)
if hasattr(best_model.named_steps['classifier'], 'feature_importances_'):
    # Récupérer les noms des variables transformées
    preprocessor = best_model.named_steps['preprocessor']
    feature_names = []
    
    # Essayer de récupérer les noms des variables à partir du transformer
    for name, transformer, columns in preprocessor.transformers_:
        if name != 'remainder':
            if hasattr(transformer, 'get_feature_names_out'):
                trans_feature_names = transformer.get_feature_names_out(columns)
            else:
                trans_feature_names = columns
            feature_names.extend(trans_feature_names)
    
    # Récupérer les importances
    importances = best_model.named_steps['classifier'].feature_importances_
    
    # Créer un DataFrame pour la visualisation
    if len(feature_names) == len(importances):
        feature_importance = pd.DataFrame({
            'feature': feature_names,
            'importance': importances
        }).sort_values('importance', ascending=False)
        
        # Afficher les 20 variables les plus importantes
        plt.figure(figsize=(12, 8))
        top_n = min(20, len(feature_importance))
        sns.barplot(x='importance', y='feature', data=feature_importance.head(top_n))
        plt.title(f'Top {top_n} variables les plus importantes')
        plt.xlabel('Importance')
        plt.ylabel('Variable')
        plt.tight_layout()
        plt.show()
        
        # Afficher le tableau des importances
        print(f"\nTop {top_n} variables les plus importantes:")
        display(feature_importance.head(top_n))

In [ ]:
## 6. Sauvegarder le modèle final

# Créer le dossier models s'il n'existe pas
os.makedirs(os.path.join('..', 'models'), exist_ok=True)

# Chemin pour sauvegarder le modèle
model_path = os.path.join('..', 'models', 'satisfaction_model.pkl')

# Sauvegarder le modèle optimisé
joblib.dump(best_model, model_path)
print(f"Modèle sauvegardé à: {model_path}")

# Tester le chargement du modèle
loaded_model = joblib.load(model_path)
y_pred_loaded = loaded_model.predict(X_test)
print(f"F1 Score du modèle chargé: {f1_score(y_test, y_pred_loaded):.4f}")

In [ ]:
## 7. Test du modèle avec des exemples

# Créer quelques exemples de passagers pour tester le modèle
test_passengers = [
    {
        'Age': 35,
        'Flight Distance': 1500,
        'Inflight wifi service': 4,
        'Departure/Arrival time convenient': 4,
        'Ease of Online booking': 5,
        'Gate location': 3,
        'Food and drink': 5,
        'Seat comfort': 4,
        'Inflight entertainment': 5,
        'On-board service': 5,
        'Leg room service': 4,
        'Baggage handling': 5,
        'Checkin service': 4,
        'Inflight service': 5,
        'Cleanliness': 5,
        'Customer Type': 'Loyal Customer',
        'Class': 'Business',
        'Type of Travel': 'Business travel'
    },
    {
        'Age': 45,
        'Flight Distance': 800,
        'Inflight wifi service': 2,
        'Departure/Arrival time convenient': 3,
        'Ease of Online booking': 3,
        'Gate location': 2,
        'Food and drink': 2,
        'Seat comfort': 2,
        'Inflight entertainment': 1,
        'On-board service': 2,
        'Leg room service': 1,
        'Baggage handling': 3,
        'Checkin service': 3,
        'Inflight service': 2,
        'Cleanliness': 3,
        'Customer Type': 'disloyal Customer',
        'Class': 'Eco',
        'Type of Travel': 'Personal Travel'
    }
]

# Convertir en DataFrame
test_df = pd.DataFrame(test_passengers)

# Prédire la satisfaction
test_predictions = loaded_model.predict(test_df)
test_probabilities = loaded_model.predict_proba(test_df)

# Afficher les résultats
for i, passenger in enumerate(test_passengers):
    print(f"\nPassager {i+1}:")
    print(f"Type de client: {passenger['Customer Type']}")
    print(f"Classe: {passenger['Class']}")
    print(f"Type de voyage: {passenger['Type of Travel']}")
    print(f"Prédiction: {'Satisfait' if test_predictions[i] == 1 else 'Non satisfait'}")
    print(f"Probabilité de satisfaction: {test_probabilities[i][1]:.2f}")

print("\n## Modélisation terminée ##")